In [1]:
from pyspark import SparkContext
sc = SparkContext('local', 'ReduceByKey App')

In [4]:
!head -n1 ../data/access.log

194.44.224.248 - - - [02/Apr/2017:06:25:23 +0200] "GET /track?action_name=%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F%20%C2%AB%D0%A4%D0%BE%D1%80%D0%BC%D1%83%D0%BB%D0%B0%C2%BB%3A%20%D0%9A%D1%80%D0%B8%D1%81%20%D0%A5%D0%B5%D0%BC%D1%81%D0%B2%D0%BE%D1%80%D1%82%20%D0%B2%20%D0%BD%D0%BE%D0%B2%D0%BE%D0%BC%20%D1%84%D0%B8%D0%BB%D1%8C%D0%BC%D0%B5%20%C2%AB%D0%93%D0%BE%D0%BD%D0%BA%D0%B0%C2%BB%20-%20%D0%B6%D1%83%D1%80%D0%BD%D0%B0%D0%BB%20Rolling%20Stone&idsite=559133ba45c0640fad7095cb&rec=1&r=575326&h=7&m=25&s=23&url=http%3A%2F%2Fwww.rollingstone.ru%2Fcinema%2Fevents%2F17928.html&urlref=https%3A%2F%2Fwww.google.com.ua%2F&_id=f67a211ada28b586&_idts=1491107124&_idvc=1&_idn=1&_refts=1491107124&_viewts=1491107124&_ref=https%3A%2F%2Fwww.google.com.ua%2F&cookie=1&res=750x1334&gt_ms=635 HTTP/1.1" 200 446 0.000 "http://www.rollingstone.ru/cinema/events/17928.html" "Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_2 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13F69 

In [8]:
# create RDD from text file
log_rdd = sc.textFile('../data/access.log').persist()

def get_platform(line):
    lower = line.lower()
    if lower.find('iphone') != -1:
        return 'iphone'
    elif lower.find('windows') != -1:
        return 'windows'
    elif lower.find('linux') != -1:
        return 'linux'
    else:
        return 'unknown'


platform_rdd = log_rdd.map(lambda line: (get_platform(line), 
                                         float(line.split()[11])))

platform_count = platform_rdd.map(lambda e: (e[0], float(e[1]))) \
                             .reduceByKey(lambda x, y: max(x, y)) \
                             .collect()

print(platform_count)  # line count by platform

# TODO: print max and average response time by platform

# TODO: print 0.9 percentile of response time by platform

[('unknown', 0.103), ('windows', 0.508), ('linux', 0.128), ('iphone', 0.135)]


In [7]:
platform_rdd.countByKey()

defaultdict(int, {'iphone': 167, 'linux': 245, 'unknown': 227, 'windows': 361})

In [17]:
for platform, count in platform_rdd.countByKey().items():
    filtered_rdd = platform_rdd.filter(
        lambda platform_time: platform_time[0] == platform)
    top = filtered_rdd.top(int(count * 0.1))
    print(top[-1])

('unknown', 0.023)
('iphone', 0.019)
('windows', 0.019)
('linux', 0.018)
